<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ngram_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-gram Language Modeling

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import torchtext.data as data

from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

In [0]:
torch.manual_seed(1)


# **Language Modeling**

**Goal:** compute a probabilty distribution over all possible sentences:


$p(W) = p(w_1, w_2, ..., w_T)$ - the probability that the sentence composed of words $(w_1, ..., w_T)$ in this order




This unsupervised learning problem can be framed as a sequence of supervised learning problems:


$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$



If we have N sentences, each of them with T words / tokens, then we want to max:

$log p(W) = \sum_{n = 1}^N \sum_{i=1}^{T} log p(w_i | w_{<i})$




# **N-gram language model**

**Goal:** estimate the n-gram probabilities using counts of sequences of n consecutive words

Given a sequence of words $w$, we want to compute

  $P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$

Where $w_i$ is the i-th word of the sequence.

$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$

**Key Idea:** We can estimate the probabilities using counts of n-grams in our dataset 


# Data Processing

# Bag of N-grams

# Train, Validation, Test Datasets

# Model

# Loss Function and Optimizer

# Training and Testing

# Analysis & Examples